In [ ]:
from selenium import webdriver
from selenium.common.exceptions import (NoSuchElementException,
                                        WebDriverException)
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import sqlite3

In [ ]:
"""demo
import sqlite3

conn = sqlite3.connect('./pokemon_name.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS pokemon_name
               (id INTEGER PRIMARY KEY, name_jp TEXT, name_tw TEXT)''')

# Assuming 'a' and 'b' are the values you want to insert for name_jp and name_tw
value_for_a = 'ピカチュウ'  # Example value for name_jp
value_for_b = '皮卡丘'    # Example value for name_tw

# Insert data into the table
cursor.execute("INSERT INTO pokemon_name (name_jp, name_tw) VALUES (?, ?)", (value_for_a, value_for_b))
conn.commit()

# Retrieve and print all rows in the table
cursor.execute("SELECT * FROM pokemon_name")
print(cursor.fetchall())

conn.close()

"""

In [ ]:
db_file = "./pokemon_name.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('''DROP TABLE IF EXISTS pokemon_name;''')
cursor.execute('''CREATE TABLE pokemon_name
               (id INTEGER PRIMARY KEY, number INT, name_jp TEXT, name_tw TEXT);''')

In [ ]:
TARGET_URL = "https://wiki.52poke.com/zh-hant/%E5%AE%9D%E5%8F%AF%E6%A2%A6%E5%88%97%E8%A1%A8%EF%BC%88%E5%9C%A8%E5%85%B6%E4%BB%96%E8%AF%AD%E8%A8%80%E4%B8%AD%EF%BC%89"
with webdriver.Chrome() as driver:
    # Init driver
    driver.implicitly_wait(2)
    driver.get(TARGET_URL)

    # Get table
    table = driver.find_element(By.CLASS_NAME, "bg-神奇宝贝百科.bd-神奇宝贝百科.hvlist.a-c.roundy")
    
    # Extract all rows and cols from the table
    rows = table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')
        row_data = [col.text for col in cols]
        print(row_data)
        if not row_data: continue

        # values we need
        number = row_data[0]
        name_jp = row_data[6]
        name_tw = row_data[2]
        name_tw = name_tw if "\n" not in name_tw else name_tw.split("\n")[0]  # raw: 鐵甲蛹\n铁甲蛹 or 烈雀

        if not number.isnumeric(): continue
        number = int(number)
        if number < 1: continue

        # Insert a row of data
        cursor.execute(
            "INSERT INTO pokemon_name (number, name_jp, name_tw) VALUES (?, ?, ?);",
            (number, name_jp, name_tw))
        conn.commit()

In [ ]:
conn.close()